In [1]:
import keras
import time
import datetime
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)

In [51]:
data = pd.read_csv('total_data.csv')
data.head().T

data = data[["id", "host_id", "host_identity_verified", "neighbourhood", "property_type", "room_type", "accommodates", "bathrooms", "bedrooms", "beds", "amenities", "price", "minimum_nights", "host_is_superhost", "has_availability", "calculated_host_listings_count", "reviews_per_month", "availability_365"]]

C:\Users\first\AppData\Local\Temp\ipykernel_4188\1355182641.py:1: DtypeWarning: Columns (1,17,19,20,28,33,34,54,62,63,66,68,69,88) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('total_data.csv')


In [30]:
le = LabelEncoder()
# data['host_identity_verified'] = data['host_identity_verified'].astype('bool')
# data['host_identity_verified_encoded'] = le.fit_transform(data['host_identity_verified'])
data['room_type_encoded'] = le.fit_transform(data['room_type'])
data['property_type_encoded'] = le.fit_transform(data['property_type'])
data['neighbourhood_encoded'] = le.fit_transform(data['neighbourhood'])
# data['has_availability'] = data['has_availability'].astype('bool')
# data['has_availability_encoded'] = le.fit_transform(data['has_availability'])
# data['host_is_superhost'] = data['host_is_superhost'].astype('bool')
# data['host_is_superhost_encoded'] = le.fit_transform(data['host_is_superhost'])
data['price'] = data['price'].replace('[\$,]', '', regex=True)
data['price'] = data['price'].astype(float)
data = data.dropna()

In [31]:
amenities = (data['amenities'].str.strip('{}').str.get_dummies(',').rename(columns= lambda x: x.strip('"')))
dataAmenities = pd.DataFrame(data['price']).join(amenities)

In [32]:
corr = dataAmenities.corr()
attrPriceCoeffs = corr['price'].drop('price')
attrPriceCoeffsVal = attrPriceCoeffs.values.reshape(-1, 1)
scaler = MinMaxScaler()
attrPriceCoeffsNorm = scaler.fit_transform(attrPriceCoeffsVal)

In [33]:
coeffs = pd.DataFrame(attrPriceCoeffsNorm)
coeffs.set_index(attrPriceCoeffs.keys(), inplace=True)
coeffs.rename(columns= lambda  x: 'price', inplace=True)
coeffs.rename(str.lower, axis='index', inplace=True)
dataAmenities.rename(columns= lambda x: x.lower(), inplace=True)
dataAmenitiesCol = dataAmenities.columns.drop('price')

for c in dataAmenitiesCol:
    sum += dataAmenities[c] * coeffs.at[c, 'price']
    dataAmenities['sum'] = sum

data['amenities_sum'] = dataAmenities['sum']

In [42]:
features = (data.drop('price', axis=1).drop('property_type', axis=1).drop('room_type', axis=1).drop('neighbourhood', axis=1)
            .drop('has_availability', axis=1).drop('host_identity_verified', axis=1).drop('id', axis=1).drop('amenities', axis=1).values)
prediction = data['price'].values
data

,id,host_identity_verified,neighbourhood,property_type,room_type,accommodates,bathrooms,bedrooms,beds,amenities,...,calculated_host_listings_count,reviews_per_month,availability_365,host_identity_verified_encoded,room_type_encoded,property_type_encoded,neighbourhood_encoded,has_availability_encoded,host_is_superhost_encoded,amenities_sum
0,17878,True,Copacabana,Condominium,Entire home/apt,5,1.0,2.0,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",...,1.0,2.16,309.0,0,1,15,24,0,0,41.764552
1,24480,True,Ipanema,Apartment,Entire home/apt,2,1.0,1.0,1.0,"{TV,""Cable TV"",Wifi,""Air conditioning"",""First ...",...,1.0,0.88,0.0,0,1,2,46,0,0,24.316472
2,25026,True,Copacabana,Apartment,Entire home/apt,3,1.0,1.0,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",...,3.0,2.15,335.0,0,1,2,24,0,0,28.619233
3,31560,True,Ipanema,Apartment,Entire home/apt,3,1.0,1.0,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",...,1.0,2.18,173.0,0,1,2,46,0,0,29.802090
4,35636,True,Ipanema,Apartment,Entire home/apt,3,1.5,1.0,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",...,1.0,2.08,144.0,0,1,2,46,0,0,43.534518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
783822,43236463,True,Barra da Tijuca,Apartment,Entire home/apt,4,1.0,1.0,1.0,"{TV,""Air conditioning"",Pool,Kitchen,""Free park...",...,1.0,2.00,86.0,0,1,2,4,0,0,25.498127
784030,43371775,True,Barra da Tijuca,Serviced apartment,Entire home/apt,2,1.0,1.0,2.0,"{TV,Wifi,""Air conditioning"",Pool,""Free parking...",...,1.0,1.00,128.0,0,1,38,4,0,0,27.930641
784033,43381083,True,Vila Isabel,Apartment,Entire home/apt,4,2.0,2.0,2.0,"{TV,""Air conditioning"",Kitchen,Elevator,""Free ...",...,2.0,3.00,150.0,0,1,2,99,0,0,20.793637
784035,43381600,True,Copacabana,Apartment,Entire home/apt,4,1.0,0.0,1.0,"{TV,Wifi,""Air conditioning"",Kitchen,Essentials...",...,1.0,2.00,60.0,0,1,2,24,0,0,16.012804


In [43]:
x_train, x_test, y_train, y_test = train_test_split(features, prediction, test_size=0.33, random_state=42)
ss = StandardScaler()
x_train = ss.fit_transform(x_train.astype(float))
x_test = ss.transform(x_test.astype(float))

In [45]:
inputs = keras.Input(shape=(15,))
outputs = keras.layers.Dense(1)(inputs)
model = keras.Model(inputs, outputs)
model.compile(optimizer=keras.optimizers.Nadam(), loss=[keras.losses.MeanSquaredError()], metrics=[keras.metrics.MeanAbsoluteError()])

start_time = time.time()
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))
print("--- %s minutes ---" % datetime.timedelta(seconds=(time.time() - start_time)))
model.summary()

Epoch 1/10
8410/8410 [==============================] - 9s 1ms/step - loss: 862158.8125 - mean_absolute_error: 368.7479 - val_loss: 894860.9375 - val_mean_absolute_error: 366.0117
Epoch 2/10
8410/8410 [==============================] - 9s 1ms/step - loss: 847218.3125 - mean_absolute_error: 361.1468 - val_loss: 880641.6250 - val_mean_absolute_error: 358.5654
Epoch 3/10
8410/8410 [==============================] - 9s 1ms/step - loss: 833491.3750 - mean_absolute_error: 353.6869 - val_loss: 867610.6250 - val_mean_absolute_error: 351.2788
Epoch 4/10
8410/8410 [==============================] - 8s 1ms/step - loss: 820996.5000 - mean_absolute_error: 346.4023 - val_loss: 855688.5625 - val_mean_absolute_error: 344.1756
Epoch 5/10
8410/8410 [==============================] - 8s 1ms/step - loss: 809557.1875 - mean_absolute_error: 339.3237 - val_loss: 844766.5000 - val_mean_absolute_error: 337.2929
Epoch 6/10
8410/8410 [==============================] - 8s 1ms/step - loss: 799004.2500 - mean_absol

In [46]:
model.predict(x_test)

4142/4142 [==============================] - 2s 579us/step


array([[  16.725334],
       [  71.250626],
       [-104.26866 ],
       ...,
       [ 232.87099 ],
       [  50.10293 ],
       [  55.511543]], dtype=float32)

In [47]:
y_test

array([200., 149., 150., ..., 802., 193., 132.])

In [ ]:
plt.plot(history.history['mean_absolute_error'])
plt.plot(history.history['val_mean_absolute_error'])
plt.title('model accuracy')
plt.ylabel('precisão')
plt.xlabel('geração')
plt.legend(['train', 'test'], loc='upper left')

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('perda')
plt.xlabel('gerações')
plt.legend(['train', 'test'], loc='upper left')

In [ ]:
y_test
model.predict(x_test)